<a href="https://colab.research.google.com/github/shawwn/gpt-2/blob/tpu/1_5B_Parameter_GPT_2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NOTE: set "Runtime -> Change runtime type" to TPU.


# fetch the code.
!git clone https://github.com/shawwn/gpt-2 -b tpu /content/gpt-2
import os
os.chdir('/content/gpt-2')

# install prerequisites.
!pip install regex
!pip install toposort
!python download_model.py 117M

# fetch a dataset. You can use your own. See http://gwern.net/GPT-2 for how to prepare one.
# This dataset consists of all Emacs Lisp code that ships with emacs as of August 2019.
# It was generated by concatenating together all .el files, separated by <|endoftext|>.
# Tab characters were converted to 8 spaces. This makes it possible for GPT-2 to learn the proper indentation of various Lisp forms.
!gsutil cp gs://sgappa-multi/gpt-2/elisp-untab-eot-v001.txt.npz .

# Note that there is currently no support for saving the trained model.
# Theoretically it might work, but you'll have to create your own storage bucket and pass in --storage_bucket gs://your-bucket/gpt-2/

# Also note that if you plan on increasing the context window on the larger models, you should pass --sample_length 1023. (It defaults to n_ctx - 1.)
# The reason is because the sampling process gets exponentially slower as sample_length increases, so a 1.5B parameter model
# with --n_ctx 2048 will default to --sample_length 2047, which would take hours to generate even one sample.

# That said, it's interesting to train a 117M with --n_ctx 2048 --sample_length 2047 and watch it generate samples 2x longer than standard GPT-2.
# (Sampling from 117M is *much* faster than the larger models, so you can get away with a larger sample_length during training.)




# 117M
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embed 768 --n_head 12 --n_layer 12

# 345M
#!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embed 1024 --n_head 16 --n_layer 24

# 1.5B
!PYTHONPATH=src python ./train.py --dataset elisp-untab-eot-v001.txt.npz --batch_size 1 --save_every 100000 --sample_every 100 --init_tpu --fresh_model --n_ctx 1024 --n_embed 1600 --n_head 24 --n_layer 48